In [1]:
# Imports
import pandas as pd
import os

In [2]:
# Helper functions
def get_table_name(file_name):
    first_occurrence = file_name.find("_")

    if first_occurrence != -1:
        second_occurrence = file_name.find("_", first_occurrence + 1)
        if second_occurrence != -1:
            return file_name[0:second_occurrence]
        else:
            return file_name.replace(".csv", "")
    else:
        return file_name.replace(".csv", "")

def merge_columns(df, name, series_to_merge):
    output = df[series_to_merge[0]]
    df.drop(columns=series_to_merge[0], inplace=True)
    series_to_merge[0]
    for series in series_to_merge[1:]:
        output.update(df[series])
        df.drop(columns=series, inplace=True)
    df[name] = output
    return df
        
        

In [3]:
# Importing tables
tables = {}
table_root = "./tables/"
count = 0
for file in os.listdir(table_root):
    file_table = pd.read_csv(table_root + file, sep=',', index_col=False)
    # Removing the empty final column that came from the trailing comma in each file
    final_col_index = len(file_table.columns) - 1
    tables[file] = file_table.iloc[:, 0:final_col_index]

In [4]:
# Exploring ways to join tables
columns = {}
for table_key in tables.keys():
    table = tables.get(table_key)
    first_column = table.columns[0]
    if not (first_column in columns):
        columns.update({first_column: 1})
    else: 
        count = columns.get(first_column)
        count = count + 1
        columns.update({first_column: count})
for column in columns.keys():
    print(column + " - " + str(columns.get(column)))

Aluminum alloy - 14
Name - 121
Metal - 12
Gas - 9
Wood - 8
Liquid - 13
Trade name - 33
Rock - 1
Material - 23
Alloy - 26
Temperature °C - 5
Soil type - 3
Mineral - 2
Rocks - 2
Semiconductor - 7
Halide - 2
Platinum alloy - 1
Temperature (°C) - 5
Alcohol - 1
Fuel - 3
Ester - 2
Ether - 2
Altitude (m) - 1


In [5]:
# Joining all the applicable tables
# The first columns of tables that should NOT be joined
exempt_first_columns = ["Temperature °C", "Altitude (m)"]
start = False
big_table = pd.DataFrame()
for table_key in tables.keys():
    if table.columns[0] not in exempt_first_columns:
        try:
            if start:
                table = tables.get(table_key)
                table_name = get_table_name(table_key)
                table = table.rename(columns={table.columns[0]: "Name"})
                table = table.astype({"Name": "string"})
                big_table = pd.merge(
                    left = big_table,
                    right = table,
                    left_on = "Name",
                    right_on = "Name",
                    suffixes = ["", "_" + table_key ],
                    how = 'outer'
                )
            else:
                start = True
                big_table = tables.get(table_key)
                big_table = big_table.astype({big_table.columns[0]: "string"})
                big_table = big_table.rename(columns={big_table.columns[0]: "Name"})
        except Exception as ex:
            print("ERROR: " + "table_key: " + table_key + ". Error Message: " + str(ex))
big_table.head()

,Name,Thermal expansion,Temperature,Refractive index (n,Specific heat capacity,Thermal conductivity,Dynamic viscosity,Thermal expansion_Thermal_Expansion_of_Pure_Metals.csv,Tensile strength [perpendicular to grain],UNS number,...,Thermal expansion_Linear_Thermal_Expansion_of_Ceramics.csv,UNS number_Electrical_Conductivity_of_Nickel_Alloys.csv,(%IACS)_Electrical_Conductivity_of_Nickel_Alloys.csv,(S/m) ×10_Electrical_Conductivity_of_Nickel_Alloys.csv,Density_Density_of_Precious_Metals.csv,UNS number_Melting_Point_of_Magnesium_Alloys.csv,Melting point_Melting_Point_of_Magnesium_Alloys.csv,Specific heat capacity_Specific_Heat_Capacity_of_Soils.csv,UNS number_Specific_Heat_Capacity_of_Nickel_Alloys.csv,Specific heat capacity_Specific_Heat_Capacity_of_Nickel_Alloys.csv
0,1050,23.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1060,23.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1100,23.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1199,23.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011,22.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
print(big_table.shape)
columns = big_table.columns
columns = list(columns)
columns.sort()
print(columns)

(62094, 456)
['(%IACS)', '(%IACS)_Electrical_Conductivity_of_Cast_Copper_Alloys.csv', '(%IACS)_Electrical_Conductivity_of_Magnesium_Alloys.csv', '(%IACS)_Electrical_Conductivity_of_Nickel_Alloys.csv', '(%IACS)_Electrical_Conductivity_of_Titanium_Alloys.csv', '(%IACS)_Electrical_Conductivity_of_Wrought_Aluminum_Alloys.csv', '(%IACS)_Electrical_Conductivity_of_Wrought_Copper_Alloys.csv', '(%IACS)_Electrical_Conductivity_of_Zinc_Alloys.csv', '(BTU/lb)', '(MJ/kg)', '(S/m) ×10', '(S/m) ×10_Electrical_Conductivity_of_Cast_Copper_Alloys.csv', '(S/m) ×10_Electrical_Conductivity_of_Magnesium_Alloys.csv', '(S/m) ×10_Electrical_Conductivity_of_Nickel_Alloys.csv', '(S/m) ×10_Electrical_Conductivity_of_Titanium_Alloys.csv', '(S/m) ×10_Electrical_Conductivity_of_Wrought_Aluminum_Alloys.csv', '(S/m) ×10_Electrical_Conductivity_of_Wrought_Copper_Alloys.csv', '(S/m) ×10_Electrical_Conductivity_of_Zinc_Alloys.csv', 'API gravity', 'Additional_Info', 'Boiling point', 'Boiling point_Boiling_Point_of_Acids.

In [7]:
# Merging columns
# Electrical_Conductivity
big_table = merge_columns(big_table, "Electrical_Conductivity_(%IACS)", ['(%IACS)', '(%IACS)_Electrical_Conductivity_of_Cast_Copper_Alloys.csv', '(%IACS)_Electrical_Conductivity_of_Magnesium_Alloys.csv', '(%IACS)_Electrical_Conductivity_of_Nickel_Alloys.csv', '(%IACS)_Electrical_Conductivity_of_Titanium_Alloys.csv', '(%IACS)_Electrical_Conductivity_of_Wrought_Aluminum_Alloys.csv', '(%IACS)_Electrical_Conductivity_of_Wrought_Copper_Alloys.csv', '(%IACS)_Electrical_Conductivity_of_Zinc_Alloys.csv'])


/tmp/ipykernel_220808/74172472.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['5.858' '5.858' '5.336' '5.8' '2.61' '1.276' '1.276' 'SIval' '2.61' '5.8'
 '5.51' '5.394' '1.45' '1.334' '1.624' '1.508' '3.248' '1.624' '1.566'
 '1.856' '1.624' '1.508' '1.856' '1.566' '1.16' '0.87' '0.754' '0.812'
 '0.986' '0.406' '0.812' '0.522' '0.267' '0.522' '0.348' '0.522' '0.348'
 '0.522' 'SIval' '5.568' 'SIval' '0.638' '0.406' '0.696']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  output.update(df[series])


In [8]:
print(big_table.shape)
columns = big_table.columns
columns = list(columns)
columns.sort()
print(columns)

(62094, 442)
['(BTU/lb)', '(MJ/kg)', 'API gravity', 'Additional_Info', 'Boiling point', 'Boiling point_Boiling_Point_of_Acids.csv', 'Boiling point_Boiling_Point_of_Alcohols.csv', 'Boiling point_Boiling_Point_of_Alkanes.csv', 'Boiling point_Boiling_Point_of_Alkenes.csv', 'Boiling point_Boiling_Point_of_Alkynes.csv', 'Boiling point_Boiling_Point_of_Bases.csv', 'Boiling point_Boiling_Point_of_Common_Substances.csv', 'Boiling point_Boiling_Point_of_Esters.csv', 'Boiling point_Boiling_Point_of_Ethers.csv', 'Boiling point_Boiling_Point_of_Gases.csv', 'Boiling point_Boiling_Point_of_Halides.csv', 'Boiling point_Boiling_Point_of_Liquids.csv', 'Boiling point_Boiling_Point_of_Oxides.csv', 'Boiling point_Boiling_Point_of_Precious_Metals.csv', 'Boiling point_Boiling_Point_of_Salts.csv', 'Brinell Hardness (HB)', 'Brinell hardness (HB)', 'Brinell hardness (HB)_Hardness_of_Cast_Copper_Alloys.csv', 'Brinell hardness (HB)_Hardness_of_Magnesium_Alloys_(Annealed).csv', 'Brinell hardness (HB)_Hardness_of_